In [1]:
import pandas as pd
from datetime import datetime, timedelta
from unidecode import unidecode
import codecs


meses = ['jan','fev','mar','abril','mai','jun','jul','ago','set','out','nov','dez']

def trim_overspace(text):
	parts = filter(lambda x: len(x) > 0,text.split(" "))
	return " ".join(parts)

def normalize_text(text):
    #.replace("'"," ") alguns municipios tem ' e as macros maravilhosas do excel vão falhar
	x = str(text).replace(".","").replace("\n","").replace(",","").upper()
	x = trim_overspace(x)
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_labels(text):
	x = str(text).replace("'"," ").replace(".","").replace("\n","").replace(",","").lower()
	x = trim_overspace(x).replace(" ","_")
	x = unidecode(x)

	if x == 'nan' or len(x) == 0 or x == '0' or x == 'nao_informado':
		return ''
	else:
		return x
    
def normalize_number(num,cast=int,error='fill',fill='0'):
	try:
		return cast(num)
	except ValueError:
		if error == 'raise':
			raise Exception(ValueError)
		elif error == 'fill':
			return normalize_number(fill,cast,'raise')
        
def date_format(date):
    return f"{date.day}/{meses[date.month-1]}"

In [2]:
casos_confirmados = pd.read_excel('Casos confirmados PR.xlsx', 'Casos confirmados', usecols='D:M')
casos_confirmados.columns = [ normalize_labels(x) for x in casos_confirmados.columns]
casos_confirmados['nome'] = casos_confirmados['nome'].apply(normalize_text)
casos_confirmados['idade'] = casos_confirmados['idade'].apply(normalize_number)
casos_confirmados['mun_resid'] = casos_confirmados['mun_resid'].apply(normalize_text)
casos_confirmados['mun_atend'] = casos_confirmados['mun_atend'].apply(normalize_text)

casos_excluidos = casos_confirmados.loc[casos_confirmados['mun_resid']=='EXCLUIR']
casos_confirmados = casos_confirmados.loc[casos_confirmados['mun_resid']!='EXCLUIR']

casos_confirmadosPR = casos_confirmados[casos_confirmados['rs'].notnull()]
print(f"Casos excluidos {len(casos_excluidos)}")
print(f"Casos confirmados PR {len(casos_confirmadosPR)}")
print(f"Casos confirmados Total {len(casos_confirmados)}")

Casos excluidos 2
Casos confirmados PR 253141
Casos confirmados Total 255696


In [3]:
obitos_confirmados = pd.read_excel('Casos confirmados PR.xlsx', 'Obitos', usecols='D:I')
obitos_confirmados.columns = [ normalize_labels(x) for x in obitos_confirmados.columns]
obitos_confirmados['nome'] = obitos_confirmados['nome'].apply(normalize_text)
obitos_confirmados['idade'] = obitos_confirmados['idade'].apply(normalize_number)
obitos_confirmados['municipio'] = obitos_confirmados['municipio'].apply(normalize_text)

obitos_excluidos = obitos_confirmados.loc[obitos_confirmados['municipio']=='EXCLUIR']
obitos_confirmados = obitos_confirmados.loc[obitos_confirmados['municipio']!='EXCLUIR']


obitos_confirmadosPR = obitos_confirmados[obitos_confirmados['rs'].notnull()]
print(f"Obitos excluidos {len(obitos_excluidos)}")
print(f"Obitos confirmados PR {len(obitos_confirmadosPR)}")
print(f"Obitos confirmados Total {len(obitos_confirmados)}")

Obitos excluidos 6
Obitos confirmados PR 5800
Obitos confirmados Total 5852


In [4]:
novos_casos = pd.read_excel('novos_casos.xlsx', dtype={'rs':str})
print(f"Total de novos casos: {len(novos_casos)}")
novos_obitos = pd.read_excel('novos_obitos.xlsx', dtype={'rs':str})
print(f"Total de novos obitos: {len(novos_obitos)}")
novos_casosPR = novos_casos.loc[novos_casos['rs'].notnull()].copy()
print(f"Total de novos casos PR: {len(novos_casosPR)}")
novos_obitosPR = novos_obitos.loc[novos_obitos['rs'].notnull()].copy()
print(f"Total de novos obitos PR: {len(novos_obitosPR)}")
novos_casosFora = novos_casos.loc[novos_casos['rs'].isnull()].copy()
print(f"Total de novos casos Fora: {len(novos_casosFora)}")
novos_obitosFora = novos_obitos.loc[novos_obitos['rs'].isnull()].copy()
print(f"Total de novos obitos Fora: {len(novos_obitosFora)}")

Total de novos casos: 1742
Total de novos obitos: 28
Total de novos casos PR: 1739
Total de novos obitos PR: 28
Total de novos casos Fora: 3
Total de novos obitos Fora: 0


In [11]:
novos_obitosPR_group = novos_obitosPR.groupby(by='mun_resid')
today = datetime.today()
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)


retroativos = novos_casosPR.loc[(novos_casosPR['data_liberacao'] <= anteontem)]

with codecs.open(f"relatorio_{(today.strftime('%d/%m/%Y').replace('/','_').replace(' ',''))}.txt","w","utf-8-sig") as relatorio:
    relatorio.write(f"{len(novos_casosPR):,} novos casos residentes divulgados".replace(',','.'))
    if len(novos_casosFora) > 0:
        relatorio.write(f"e {len(novos_casosFora):,} não residente{'s' if len(novos_casosFora) > 1 else ''} ".replace(',','.'))
    relatorio.write(f"no PR. Sendo:\n")
    relatorio.write(f"Em {today.strftime('%d/%m/%Y')}: {len(novos_casosPR.loc[(novos_casosPR['data_liberacao'] > anteontem)])} novos casos novos confirmados.\n")
    relatorio.write(f"{len(retroativos)} casos retroativos confirmados do período de {retroativos.iloc[0]['data_liberacao'].strftime('%d/%m/%Y')} à {retroativos.iloc[-1]['data_liberacao'].strftime('%d/%m/%Y')}.\n")

    relatorio.write(f"{len(casos_confirmadosPR)+len(novos_casosPR):,} casos confirmados residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(casos_confirmados)+len(novos_casos):,} total geral.\n\n".replace(',','.'))
    relatorio.write(f"{len(novos_obitosPR):,} Óbitos residentes do PR:\n".replace(',','.'))
    for municipio, obitos in novos_obitosPR_group:
        relatorio.write(f"{len(obitos):,} {municipio}\n".replace(',','.'))
    if len(novos_obitosFora) > 0:
        relatorio.write(f"{len(novos_obitosFora):,} Óbito{'s' if len(novos_obitosFora) > 1 else ''} não residente{'s' if len(novos_obitosFora) > 1 else ''} do PR.\n\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmadosPR)+len(novos_obitosPR):,} óbitos residentes do PR.\n".replace(',','.'))
    relatorio.write(f"{len(obitos_confirmados)+len(novos_obitos):,} total geral.\n\n".replace(',','.'))
    for index, row in novos_obitosPR.iterrows():
        relatorio.write(f"{row['sexo']} {row['idade']} {row['mun_resid']} {row['rs']} {row['data_cura_obito'].day}/{meses[row['data_cura_obito'].month-1]}\n")
    relatorio.write('\n')
    
    novos_casosPR['month'] = novos_casosPR.apply(lambda x: x['data_liberacao'].month, axis=1)
    for group, value in novos_casosPR.groupby(by='month'):
        relatorio.write(f"{meses[int(group)-1]}: {len(value)}\n")
    relatorio.write('\n')
    for group, value in novos_casosPR.groupby(by='data_liberacao'):
        relatorio.write(f"{group.strftime('%d/%m/%Y') }: {len(value)}\n")

In [12]:
with codecs.open(f"relatorio_{(today.strftime('%d/%m/%Y').replace('/','_').replace(' ',''))}.txt","r","utf-8-sig") as relatorio:
    print(relatorio.read())

1.739 novos casos residentes divulgadose 3 não residentes no PR. Sendo:
Em 23/11/2020: 707 novos casos novos confirmados.
1032 casos retroativos confirmados do período de 18/11/2020 à 20/11/2020.
254.880 casos confirmados residentes do PR.
257.438 total geral.

28 Óbitos residentes do PR:
1 APUCARANA
3 CASCAVEL
1 CORNELIO PROCOPIO
10 Curitiba
1 FIGUEIRA
3 GUARAPUAVA
1 IPIRANGA
1 MAMBORE
1 MARINGA
1 MEDIANEIRA
1 NOVA TEBAS
1 PINHAIS
1 PONTA GROSSA
1 RIO BOM
1 SAO JOSE DOS PINHAIS
5.828 óbitos residentes do PR.
5.880 total geral.

M 65 MAMBORE 11 16/nov
F 89 Curitiba 02 17/nov
F 86 IPIRANGA 03 18/nov
M 60 PINHAIS 02 16/nov
M 57 MEDIANEIRA 09 22/nov
M 83 NOVA TEBAS 22 23/nov
F 80 Curitiba 02 13/nov
F 90 MARINGA 15 21/nov
F 73 CASCAVEL 10 20/nov
M 66 Curitiba 02 16/nov
M 76 PONTA GROSSA 03 18/nov
M 82 CORNELIO PROCOPIO 18 21/nov
M 62 GUARAPUAVA 05 9/nov
M 86 RIO BOM 16 23/nov
M 72 FIGUEIRA 19 22/nov
M 84 Curitiba 02 14/nov
M 64 Curitiba 02 17/nov
F 68 Curitiba 02 17/nov
F 64 CASCAVEL 10 20